# Preliminaries

#### Imports

In [1]:
import os
import glob
import PIL
import numpy as np
import math
from xml.etree import ElementTree
from contextlib import contextmanager
import threading
from google.colab import output
from google.colab import drive

ModuleNotFoundError: No module named 'google'

#### Installing utilities

In [ ]:
#install Go

!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go get -u github.com/gopherdata/gophernotes
!cp ~/go/bin/gophernotes /usr/bin/
!mkdir /usr/local/share/jupyter/kernels/gophernotes
!cp ~/go/src/github.com/gopherdata/gophernotes/kernel/* \
       /usr/local/share/jupyter/kernels/gophernotes

# then refresh notebook!

In [ ]:
#install AuGoment
! git clone https://github.com/lootag/ImageAuGomentationCLI.git
os.chdir('ImageAuGomentationCLI' + os.sep + 'src')
! make
os.chdir('/content')

In [ ]:
! pip install mean_average_precision

#### Import Repository

In [ ]:
! git clone https://github.com/guglielmoG/windspeed.git

#### Definitions

In [ ]:
from windspeed.utils import *

repo = 'windspeed'
detection_classes = ['flag']

# TO_DELETE
Possiamo mettere legacy code che non ha senso stia da altre parti ma che potrebbe essere utile in futuro

In [ ]:
#use to rename pictures
with cwd('windspeed', 'flags'):
    # get all annots
    ann = set(os.listdir('Annotations'))
    c = 0
    for f in os.listdir('images'):
        img_name, ext = os.path.splitext(f)
        if os.path.exists(join_path('Annotations', img_name+'.xml')):
            os.rename(join_path('Annotations', img_name+'.xml'), join_path('Annotations', str(c) + 'flag.xml'))
            os.rename(join_path('images', f), join_path('images', str(c)+'flag'+ext))
            ann.remove(img_name+'.xml')
        else:
            os.rename(join_path('images', f), f)
            print(f)
        c += 1
            
    print('No images for ', ann)

In [ ]:
#used to debug augoment, identify images for which creates problems
with cwd(repo):
    ! git checkout .
    ! git clean -fd

with cwd(repo, 'data', 'flags'):
    os.mkdir('temp')
    os.mkdir(join_path('temp', 'Images'))
    os.mkdir(join_path('temp', 'Annotations'))
    c = 0
    for f in os.listdir('Images'):
        img_name,_ = os.path.splitext(f)
        os.rename(join_path('Annotations', img_name+'.xml'), join_path('temp', 'Annotations', img_name+'.xml'))
        os.rename(join_path('Images', f), join_path('temp', 'Images', f))
        try:
            with cwd('temp'):
                ! augoment -exclusion_threshold=1 -batch_size=1 -blur=2 >good 2>bad
                ! echo {f} > boh
                ! grep panic bad >> boh
                tt = ! grep panic bad
                if len(tt) != 0:
                    ! cat boh >> out_good
                    ! echo "" >> out_good
        except Exception as e:
            print(f, str(e))
        finally:
            os.remove(join_path('temp', 'Annotations', img_name+'.xml'))
            os.remove(join_path('temp', 'Images', f))
        print(c)
        c += 1

# END_TO_DELETE

# Flag Detection

Folder structure:  
-train  
-validation  
-test  

In each folder one can find images and annotations, separated.

## Data Preprocessing

### Folder Structure Creation
Divide in training, validation and test set, with approximately 75%, 15% and 10% respectively.

In [ ]:
np.random.seed(3456)

def make_augoment_dir(name):
    os.mkdir(name)
    os.mkdir(join_path(name, 'Images'))
    os.mkdir(join_path(name, 'Annotations'))
    
with cwd(repo, 'data', 'flags'):
    make_augoment_dir('train')
    make_augoment_dir('valid')
    make_augoment_dir('test')
    imgs = os.listdir('Images')
    n = len(imgs)
    #use random indexes to shuffle the images
    idx = np.arange(n)
    np.random.shuffle(idx)
    for i in range(n):
        img = imgs[idx[i]]
        img_name, _ = os.path.splitext(img)
        if i < n * 0.75:
            os.rename(join_path('Images', img), join_path('train', 'Images', img))
            os.rename(join_path('Annotations', img_name + '.xml'), join_path('train', 'Annotations', img_name + '.xml'))
        elif i < n * 0.9:
            os.rename(join_path('Images', img), join_path('valid', 'Images', img))
            os.rename(join_path('Annotations', img_name + '.xml'), join_path('valid', 'Annotations', img_name + '.xml'))
        else:
            os.rename(join_path('Images', img), join_path('test', 'Images', img))
            os.rename(join_path('Annotations', img_name + '.xml'), join_path('test', 'Annotations', img_name + '.xml'))
    os.rmdir('Images')
    os.rmdir('Annotations')

### Augmentation
NB: if one would like to obtain different blur values for a given image (i.e. produce say 3 new blurred images out of every image, with different blur values), they would need to rename previosly created blurred images, because they are overwritten.

Convert images to jpg for augoment to work properly, then run augoment.

In [ ]:
convert_to_jpg(join_path(repo,'data','flags','train', 'Images'))
convert_to_jpg(join_path(repo,'data','flags','valid', 'Images'))
convert_to_jpg(join_path(repo,'data','flags','test', 'Images'))

with cwd(join_path(repo,'data','flags','train')):
    ! augoment -blur=2
    
with cwd(join_path(repo,'data','flags','valid')):
    ! augoment -blur=2

### YOLO Preprocessing

Clone the repo, and compile code with CUDA.

In [ ]:
! git clone https://github.com/pjreddie/darknet.git

with cwd('darknet'):
    ! sed -i 's/GPU=0/GPU=1/' Makefile
    ! sed -i 's/CUDNN=0/CUDNN=1/' Makefile
    ! make
    
#copy needed files
os.rename(join_path('darknet', 'darknet'), join_path(repo, 'darknet'))
os.mkdir(join_path(repo, 'backup'))
os.mkdir(join_path(repo, 'results'))

Convert annotations from xml to YOLO format. Additionally, create input txt files containing images paths, as required by YOLO.

In [ ]:
def collect_img_path(in_dir, out_path):
    l = glob.glob(join_path(in_dir, '*[!.txt]'))
    with open(out_path, 'w') as w:
        w.write('\n'.join(l))
        
#converting annotations
with cwd(repo, 'data', 'flags'):
    for f in os.listdir(): #train, valid, test
        if os.path.isdir(f) and f in ['train', 'valid']:
            for xml in os.listdir(join_path(f, 'AugmentedAnnotations')):
                convert_annot_yolo(join_path(f, 'AugmentedAnnotations', xml), detection_classes, join_path(f, 'AugmentedImages'))

#creating input paths to images
with cwd(repo):
    collect_img_path(join_path('data','flags','train','AugmentedImages'), join_path('data','flags','train.txt'))
    collect_img_path(join_path('data','flags','valid','AugmentedImages'), join_path('data','flags','valid.txt'))

The structure of _cfg/flag.data_ is as follows
```
  classes = 1
  train  = <path-to-data>/train.txt
  valid  = <path-to-data>/valid.txt
  names = <path-to-data>/flag.names
  backup = /mydrive/yolov3
```
_flag.names_ should contain the list of classes, one per line.  

Download weights for the classifier (Darknet-53)

In [ ]:
! wget https://pjreddie.com/media/files/darknet53.conv.74
os.rename('darknet53.conv.74', join_path(repo,'cfg','darknet53.conv.74'))

Edit the cfg file for _yolo-v3_. Need to set adeguate `batch` (number of images per training step) and `subdivisions` (further subdivide the batch to speedup training) values according to the hardware at disposal. Then need to update `classes` in yolo layer to match the number of classes that needs to be predicted. Finally, adjust `filters` amount in convolutional layer prior to yolo layer, to match the updated number of classes, according to the formula:
```
filters = (classes + 5)*3
```
The file _yolo-v3.cfg_ provided is already setup in such a way.  

### Training

In [ ]:
drive.mount('/content/gdrive')
! ln -s /content/gdrive/My\ Drive/ /mydrive
! mkdir -p "/mydrive/yolov3"

In [ ]:
def clean():
    output.clear()

#clean every minute, for 12 hours
for x in range(20,1*60*60*12,60):
    timer = threading.Timer(x, clean)
    timer.start()

with cwd(repo):
    ! chmod 755 darknet
    ! ./darknet detector train cfg/flag.data cfg/yolov3.cfg cfg/darknet53.conv.74 -dont_show

### Inference

We start by importing our trained model into opencv. To predict we can use`predict_yolo()`, which takes as input the network, and image, and predicts and returns the bounding boxes. They are encoded with one bounding box per row, represented as `center_x, center_y, w, h, confidence` where `center_x, center_y` are the center pixels of the bounding box, `w` and `h` are half of the width and half of the height respectively. Finally, `confidence` stands for the prediction confidence of the network.

At this stage we are interested in assessing the quality of our model out of sample, computing the mAP. To do that we use a wrapper function that takes a generic model, custom predict function for that model, class labels and image folder, and computes the metric of interest. As for the optional parameters, we are asking to compute mAP according to both pascal_voc definiton and COCO. Finally, `net_input` refers to the image input size fo YOLO, as specified in the config file.

In [ ]:
! cp /mydrive/yolov3/yolov3.backup /content
os.rename('yolov3.backup', join_path(repo, 'cfg', 'volov3.weights'))

with cwd(repo, 'cfg'):
    net = cv2.dnn.readNet("volov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = net.getUnconnectedOutLayersNames()

mAP = evaluate_model(net, predict_yolo, detection_classes, mdl_type='detection', 
                     path=join_path(repo,'data','flags','test'),
                     mAP_type='both', net_input_w=608, net_input_h=608)

### SSD Preprocessing
Let's choose the model and the hyperparameters. For SSD we will use a `ssd_mobilenet_v2`, which offers good tradeoff between speed and accuracy and it is specifically designed to run on mobile device. Along with the model let's also set the hyper parameters associated with the training.

In [ ]:
num_steps = 40000 
num_eval_steps = 50
MODEL = 'ssd_mobilenet_v2_coco_2018_03_29'
pipeline_file = 'ssd_mobilenet_v2_coco.config'
batch_size = 12
num_classes = 1

First of all we need to clone the repository of TensorFlow model and install all the required lbraries.

In [ ]:
os.mkdir(os.path.join(repo, 'OD_SSD'))
with cwd(repo, 'OD_SSD'):
  ! git clone --quiet https://github.com/tensorflow/models.git

! pip install tf_slim
! pip install tf-models-official
! apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
! pip install -q Cython contextlib2 pillow lxml matplotlib
! pip install -q pycocotools
! pip install tf-models-official
! pip install lvis

with cwd(repo, 'OD_SSD','models','research'):  
  ! protoc object_detection/protos/*.proto --python_out=.
  os.environ['PYTHONPATH'] += ':/content/windspeed/OD_SSD/models/research/:/content/windspeed/OD_SSD/models/research/slim/'
  ! python object_detection/builders/model_builder_test.py

In order to use TensorFlow we first need to encode our data in the `.tfrecord` format. In order to do so we first need to convert all the annotations in a single `.csv` file.

In [ ]:
# create the .csv files
for f in ['train','valid']:
  image_path = os.path.join(repo, 'data/flags',f,'AugmentedAnnotations')
  xml_df = xml_to_csv(image_path)
  xml_df.to_csv(os.path.join(repo,'OD_SSD', f + '_labels.csv'), index=None)

# create the .tfrecord files
os.mkdir(os.path.join(repo,'OD_SSD','tfrecord'))

with cwd(repo,'OD_SSD','tfrecord'):
  ! wget https://raw.githubusercontent.com/fllay/totoro/master/tfrecord/generate_tfrecord.py


In [ ]:
# !!!! before running this command it is necessary to modify the generate_tfrecord.py
# I have kept this form because I am a little bit confuse on how to put it in utils.py 
with cwd(repo, 'OD_SSD','tfrecord'):
  ! python generate_tfrecord.py --csv_input=/content/windspeed/OD_SSD/train_labels.csv  --output_path=train.tfrecord --image_dir=/content/windspeed/data/flags/train/AugmentedImages 
  ! python generate_tfrecord.py --csv_input=/content/windspeed/OD_SSD/val_labels.csv  --output_path=val.tfrecord --image_dir=/content/windspeed/data/flags/valid/AugmentedImages


We then dowload from TensorFlow model zoo a pretrained model as a starting point to train the model on our dataset in the pretrained model folder.


In [ ]:
import os
import shutil
import glob
import urllib.request
import tarfile

#with cwd(repo, 'OD_SSD','models','research'):

MODEL = 'ssd_mobilenet_v2_coco_2018_03_29'

MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = os.path.join (os.getcwd(), repo, 'OD_SSD/pretrained_model')

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

# we the path of the pretrained model to the following variable
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")

# and create the label map, to map each the prediction with the lable
label_map_v1('Flag')

#Then we assigned to a variable of the .tfrecord and of the .pbtxt to make the notation clearer
test_record_fname = os.path.join(os.getcwd(), repo, 'OD_SSD/tfrecord/val.tfrecord')
train_record_fname = os.path.join(os.getcwd(), repo, 'OD_SSD/tfrecord/train.tfrecord')
label_map_pbtxt_fname = os.path.join(os.getcwd(), repo, 'OD_SSD/label_map.pbtxt')

Then we need to configure the pipeline using the following hyper paramters

In [ ]:
num_steps = 40000 
num_eval_steps = 50
pipeline_file = 'ssd_mobilenet_v2_coco.config'
batch_size = 12
num_classes = 1

pipeline_fname = os.path.join(os.getcwd(), repo, 'OD_SSD/models/research/object_detection/samples/configs/', pipeline_file)

# by calling this function we can configure the pipeline according to the selected hyperparameter
configuring_pipeline(pipeline_fname, train_record_fname, test_record_fname, label_map_pbtxt_fname, batch_size, num_step)

### Training

In [ ]:
os.mkdir(os.path.join(repo,'OD_SSD','training_model'))
model_dir = os.path.join(repo,'OD_SSD','training_model')
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

### Saving the model

In [ ]:
output_directory = os.path.join(os.getcwd(), repo, 'OD_SSD/fined_tuned_model' ) 
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}
pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")

from google.colab import files
files.download(pb_fname)

### Running the model

In [ ]:
sys.path.append('/content/windspeed/OD_SSD/models/research')

# need to check
import pathlib
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

PATH_TO_CKPT = os.path.join(os.getcwd(),repo, 'OD_SSD/fined_tuned_model/frozen_inference_graph.pb')
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(os.getcwd(),repo, 'OD_SSD/label_map.pbtxt')

mAP = evaluate_model(PATH_TO_CKPT, predict_fn_ssd, detection_classes, mdl_type='detection', 
                     path=join_path(repo,'data','flags','test'), PATH_TO_LABELS = PATH_TO_LABELS)

### DELETE
This code is to visually check whether the model work

In [ ]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                              feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  
  return output_dict

TEST_IMAGE_PATHS = ['/content/windspeed/data/flags/test/Images/100flag.jpg']

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np, detection_graph)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)